In [2]:
##Estimación de ingreso promedio trimestral por decil de hogares
#Se borra todo lo que se encuentra en el entorno

#rm(list=ls())
# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,srvyr,showtext,gt)
font_add_google("Poppins", "pop")
showtext_auto()

Cargando paquete requerido: pacman



In [3]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

#### Elementos básicos para contar con los datos

In [4]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin<-"_ns_concentradohogar_csv.zip"

In [5]:
years<-c("2016", "2018", "2020", "2022", "2024")
years

[1] "2016" "2018" "2020" "2022" "2024"

In [6]:
# Descargar los archivos, descomprimirlos y renombrarlos
options(timeout = 300) 
for (i in 1:length(years)) {
  url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
  destfile <- paste0("data/", years[i], fin)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = "data")

  # Renombrar el archivo descomprimido a "viviendas_xxxx.csv"
  file.rename(
    from = "data/concentradohogar.csv",
    to = paste0("data/concentradohogar_", years[i], ".csv")
  )
}

In [9]:
url_inpc<-"https://www.inegi.org.mx/contenidos/programas/inpc/2018a/datosabiertos/conjunto_de_datos_inpc_indicador_mensual_csv.zip"

download.file(url_inpc, destfile = "data/inpc/inpc.zip")
#Descomprimir el archivo
unzip("data/inpc/inpc.zip", exdir = "data/inpc")

In [10]:
#Leer INPC
inpc <- read_csv("data/inpc/conjunto_de_datos/conjunto_de_datos_inpc_mensual.csv",show_col_types = FALSE) %>%
  janitor::clean_names()%>%
  #Extraer year y month
  mutate(year=substr(fecha,1,4),
         month=substr(fecha,6,7))%>%
         filter(concepto=="Índice nacional de precios al consumidor (mensual), Resumen, Subíndices subyacente y complementarios, Precios al Consumidor (INPC)")%>%
         select(year,month,valor)%>%
         filter(year>=2016 & year<=2024)%>%
         #Obtener el promedio anual
group_by(year)%>%
summarise(valor=mean(valor))%>%
#Cambiar la base a 2024=100
mutate(inpcbase=valor/valor[year=="2024"]*100)%>%
mutate(year=as.integer(year))

inpc

year,valor,inpcbase
<int>,<dbl>,<dbl>
2016,90.12792,66.57178
2017,95.57296,70.59368
2018,100.25542,74.05232
2019,103.90067,76.74483
2020,107.43000,79.35172
2021,113.54192,83.86621
2022,122.50750,90.48852
2023,129.27967,95.49069
2024,135.38458,100.00000


In [11]:
paths <- list.files("data",
                    pattern = "^concentradohogar_\\d{4}\\.csv$",
                    full.names = TRUE)
paths

[1] "data/concentradohogar_2016.csv" "data/concentradohogar_2018.csv"
[3] "data/concentradohogar_2020.csv" "data/concentradohogar_2022.csv"
[5] "data/concentradohogar_2024.csv"

#### Creación de función de estimación

In [12]:
estima_ingprom <- function(path, variable = "ingreso_prom") {

  message(path)

  # --- 1) Cargar datos 
  datos <- read_csv(path, show_col_types = FALSE) %>%
    arrange(ing_cor, folioviv, foliohog) %>%
    mutate(
      c0 = lag(cumsum(factor), default = 0),
      c1 = c0 + factor
    ) %>%
    {
      # --- 2) Tamaño de decil 
      tam_dec <- floor(sum(.$factor, na.rm = TRUE) / 10)

      # --- 3) Partir el peso por fronteras; decil 10 absorbe remanente (right = Inf)
      rowwise(.) %>%
        mutate(.split = list({
          d_ini <- pmax(1L, floor(c0 / tam_dec) + 1L)
          k <- d_ini:10L
          left  <- (k - 1L) * tam_dec
          right <- ifelse(k == 10L, Inf, k * tam_dec)

          tibble(
            decil_hogar = k,
            factor_part = pmax(0, pmin(c1, right) - pmax(c0, left))
          ) %>% filter(factor_part > 0)
        })) %>%
        ungroup() %>%
        unnest(.split) %>%
        mutate(factor = factor_part) %>%
        select(-factor_part, -c0, -c1)
    } 

  # --- 4) Diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm, strata = est_dis, weights = factor)

  # --- 5) Estimación por decil
  por_decil <- mydesign %>%
    group_by(decil_hogar) %>%
    summarise(
      hogares  = survey_total(vartype = "cv"),
      ing_prom = survey_mean(ing_cor, vartype = "cv"),
      .groups  = "drop"
    ) %>%
    arrange(decil_hogar)

  # --- 6) Estimación total
  total_row <- mydesign %>%
    summarise(
      hogares  = survey_total(vartype = "cv"),
      ing_prom = survey_mean(ing_cor, vartype = "cv")
    ) %>%
    mutate(decil_hogar = "Total")

  # --- 7) Unir Total + deciles y agregar año
  out <- bind_rows(
    total_row,
    por_decil %>% mutate(decil_hogar = as.character(decil_hogar))
  ) %>%
    mutate(
      decil_hogar = factor(decil_hogar, levels = c("Total", as.character(1:10))),
      year = parse_number(basename(path))
    ) %>%
    arrange(decil_hogar)

  if (variable == "ingreso_prom") return(out)
  stop("Variable no reconocida. Use 'ingreso_prom'.")
}

#### Estimación para 2024

In [13]:
estima_ingprom("data/concentradohogar_2024.csv", "ingreso_prom")%>%
    left_join(inpc)%>%
    mutate(ing_prom_real=ing_prom/inpcbase*100)

data/concentradohogar_2024.csv

Joining with `by = join_by(year)`


hogares,hogares_cv,ing_prom,ing_prom_cv,decil_hogar,year,valor,inpcbase,ing_prom_real
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
38830230,0.003756163,77863.84,0.0077779855,Total,2024,135.3846,100,77863.84
3883023,0.017117239,16795.33,0.0043999644,1,2024,135.3846,100,16795.33
3883023,0.015118327,28296.82,0.0013549261,2,2024,135.3846,100,28296.82
3883023,0.015483896,36844.61,0.0010042583,3,2024,135.3846,100,36844.61
3883023,0.015567541,45244.52,0.0008862927,4,2024,135.3846,100,45244.52
3883023,0.016023380,54307.66,0.0008154898,5,2024,135.3846,100,54307.66
3883023,0.016018913,64599.75,0.0007831177,6,2024,135.3846,100,64599.75
3883023,0.016235631,77450.79,0.0009039882,7,2024,135.3846,100,77450.79
3883023,0.015832645,95291.21,0.0009823372,8,2024,135.3846,100,95291.21


#### Prueba para 2020

In [14]:
estima_ingprom("data/concentradohogar_2020.csv", "ingreso_prom")%>%
    left_join(inpc)%>%
    mutate(ing_prom_real=ing_prom/inpcbase*100)

data/concentradohogar_2020.csv

Joining with `by = join_by(year)`


hogares,hogares_cv,ing_prom,ing_prom_cv,decil_hogar,year,valor,inpcbase,ing_prom_real
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
35749659,0.003329212,50309.313,0.0079605765,Total,2020,107.43,79.35172,63400.40
3574965,0.017625361,9937.778,0.0042294723,1,2020,107.43,79.35172,12523.71
3574965,0.014001495,16862.471,0.0013949381,2,2020,107.43,79.35172,21250.29
3574965,0.014889921,22273.714,0.0009942438,3,2020,107.43,79.35172,28069.60
3574965,0.014902130,27558.387,0.0008309191,4,2020,107.43,79.35172,34729.41
3574965,0.015454630,33366.628,0.0008485271,5,2020,107.43,79.35172,42049.03
3574965,0.015226261,40107.594,0.0008106754,6,2020,107.43,79.35172,50544.07
3574965,0.014909880,48670.216,0.0008703591,7,2020,107.43,79.35172,61334.79
3574965,0.015416104,60597.789,0.0010821792,8,2020,107.43,79.35172,76366.07
